In [1]:
import os
import requests
from datetime import datetime, timedelta
import time
import warnings
import pandas as pd
import numpy as np
from pycorenlp import StanfordCoreNLP
import random
from IPython.display import clear_output

warnings.filterwarnings('ignore')

def get_sentiment(text):
    """Returns a float from Stanford NLP sentimentValue results"""
    result = nlp.annotate(text, properties={
                       'annotators': 'sentiment',
                       'outputFormat': 'json',
                       'timeout': 5000,
                   })
    return np.mean([int(i['sentimentValue']) for i in result['sentences']])

"""
Paste this in terminal window to start server:

cd stanford-corenlp-full-2018-10-05
java -mx6g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -timeout 5000

"""

#Connect to running Stanford NLP server
nlp = StanfordCoreNLP('http://localhost:9000')

#Read file, get number of headlines/tweets
df = pd.read_pickle('stock_prediction_data2.p')
df['num_headlines'] = df['NewsHeadlineList'].str.len()
df['num_tweets'] = df['TweetList'].str.len()


headline_sent = []
twitter_sent = []
counter = 0

#Loop over data to get sentiment score for each headline/tweet that day, then take the mean as that days sentiment
for i in df.values.tolist():
    today_headline_sent = []
    today_twitter_sent = []
    headlines, tweets = i[2], i[7]
    for h in headlines:
        try:
            score = get_sentiment(h)
            today_headline_sent.append(score)
        except:
            pass
        
    #too many tweets, only taking a random 10% sample
    sample_size = int(len(tweets) * 0.1)
    tweet_samples = random.sample(tweets, sample_size)
    for t in tweet_samples:
        try:
            score = get_sentiment(t)
            today_twitter_sent.append(score)
        except:
            pass
        
    todays_headline_sent = np.mean(today_headline_sent)
    todays_twitter_sent = np.mean(today_twitter_sent)
    headline_sent.append(todays_headline_sent)
    twitter_sent.append(todays_twitter_sent)
    clear_output(wait=True)
    print (counter / len(df))
    counter += 1
    
df['headline_sentiment'] = headline_sent
df['twitter_sentiment'] = twitter_sent

0.9969230769230769
